# Weekly Challenge 14

*Original URL* https://community.alteryx.com/t5/Weekly-Challenge/Challenge-14-Warehouse-Distribution/td-p/36743 and [**My Alteryx Approach**](https://github.com/dsmdavid/Alteryx-Weekly-Challenge/tree/master/submitted/sub_Challenge%2314)

## Brief
Here is a new challenge for this week, it is a two part challenge so next week’s challenge will be a continuation.  

### Retail distribution:

This week we are looking at a retail distribution analysis.  We need to allocate products from the warehouse to stores based on priority.  I have seen this challenge solved both with and without the use of an iterative macro.

#### The use case:

A retail chain has 25 stores carrying variety of items.  Not every store carries the same items and each has its own level of prioritization within the chain and different required stock levels.  There is a central warehouse that contains all of the available items.

 
The objective is to distribute items from the warehouse to each store, filling the available stock at each store in order of the store's priority.

In [1]:
import pandas as pd
import os

## Approach I want to follow:
1. Read the files, combine priority and quantities.
1. Create a function that distributes items from the warehouse.
1. Fulfill requirments of the stores in the priority order.
1. Verify output.

## 1. Read and combine

In [2]:
#Read the files
os.chdir(os.path.join(os.getcwd(), '14_files'))

In [3]:
os.listdir()

['01_storesPriority.csv',
 '02_storesRequirements.csv',
 '03_warehouseContents.csv',
 '04_solution.csv']

In [4]:
#Load the data:
input_dfStores = pd.read_csv("./01_storesPriority.csv", encoding="latin")     
input_dfRequirements = pd.read_csv("./02_storesRequirements.csv", encoding = "latin")
input_dfWarehouse = pd.read_csv("./03_warehouseContents.csv", encoding = "latin")

In [5]:
#Sort Stores by Priority
storesInOrder = list(input_dfStores.sort_values(by=['Priority'])['Store'])

In [6]:
#Activate Warehouse
input_dfWarehouse.set_index('Item', inplace=True )


## 2. Create functions

In [7]:
def shipping(row):
    if row['Required'] <= row['Count']:
        return row['Required']
    else:
        return row['Count']
    

In [8]:
def fulfill_store(store_str, input_dfWarehouse):
    #Process one store:
    #1. Get items required
    dfItems = input_dfRequirements[input_dfRequirements['Store']==store_str][['Item','Required']]
    dfItems.set_index('Item', inplace=True)
    #2. Check against warehouse
    dfTransient = input_dfWarehouse.join(dfItems, how='left').fillna(0)
    #3. Get the min(required, in warehouse)
    dfTransient['Assigned'] = dfTransient.apply(lambda row:shipping(row), axis=1)
    #4. Return the store.
    dfStore = dfTransient.copy()[['Required','Assigned','Warehouse']]
    dfStore['Store'] = store_str
    #5. Return the updated warehouse
    dfTransient['Count'] = dfTransient['Count'] - dfTransient['Assigned']
    input_dfWarehouse = dfTransient.drop(labels =['Required', 'Assigned'],axis=1).copy()

    return dfStore, input_dfWarehouse


In [9]:
#Starting Warehouse
input_dfWarehouse

,Warehouse,Count
Item,,
1,Main,824
2,Main,1145
3,Main,1354
4,Main,916
5,Main,1120
6,Main,635
7,Main,1056
8,Main,824
9,Main,616


## 3. Process stores

In [10]:
#Process all stores
list_df = []
for i in storesInOrder:
    new_store, input_dfWarehouse = fulfill_store(store_str = i, input_dfWarehouse = input_dfWarehouse)
    list_df.append(new_store)
    

In [11]:
#ending warehouse
input_dfWarehouse

,Warehouse,Count
Item,,
1,Main,73.0
2,Main,288.0
3,Main,662.0
4,Main,159.0
5,Main,110.0
6,Main,0.0
7,Main,0.0
8,Main,0.0
9,Main,0.0


## 4. Compare to existing results

In [12]:

#Load existing results
solution_df = pd.read_csv("./04_solution.csv", encoding = "latin")

In [13]:
# At this point, the results exist as a dataframe per store with all the potential items with the corresponding assignments.
# The output to compare against is a single dataframe with all stores with only the items that were required

#1. Concatenate all the stores
updated_stores = pd.concat(list_df,axis=0)

updated_stores['Item'] = updated_stores.index

output_df = input_dfStores[['Store','Priority','City','State']].merge(updated_stores, left_on='Store', right_on='Store')

#2. Keep only items with a requirement, change the order of the columns to match the order in the solutions and match the dtypes
test = output_df[output_df['Required']!=0][solution_df.columns].astype({'Required':'int64', 'Assigned':'int64'})
test.reset_index(drop=True, inplace=True)

In [14]:
test.equals(solution_df)

True